In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold  
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler 
import lightgbm as lgbm
from catboost import CatBoostClassifier 
from xgboost import XGBClassifier

from pycaret.classification import *

In [9]:
train = pd.read_csv('lgbm_train_df.csv') 
test = pd.read_csv('lgbm_test_df.csv') 

In [11]:
train = train.drop(columns=['n_id','title','content','title_tokenized','content_tokenized']) 

In [15]:
test = test.drop(columns=['n_id','id','title','content','title_tokenized','content_tokenized'])

In [17]:
clf = setup(data = train, target = 'info')

Setup Succesfully Completed!


,Description,Value
0,session_id,2743
1,Target Type,Binary
2,Label Encoded,"0: 0, 1: 1"
3,Original Data,"(118745, 64)"
4,Missing Values,False
5,Numeric Features,50
6,Categorical Features,13
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


After completing the setup, the compare_models() function trains all the models in the model library and scores them using stratified cross validation for metric evaluation. 

In [18]:
best_7 = compare_models(sort = 'Accuracy', n_select = 7)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Extreme Gradient Boosting,0.9904,0.9995,0.9833,0.9922,0.9878,0.9798,0.9799,36.5459
1,CatBoost Classifier,0.9901,0.9995,0.9829,0.9921,0.9875,0.9794,0.9794,14.8942
2,Light Gradient Boosting Machine,0.9865,0.9991,0.9765,0.9893,0.9828,0.9718,0.9718,2.2457
3,Extra Trees Classifier,0.9813,0.9992,0.9568,0.9956,0.9758,0.9605,0.9610,2.7548
4,Random Forest Classifier,0.9797,0.9976,0.9558,0.9925,0.9738,0.9572,0.9576,0.8061
5,Decision Tree Classifier,0.9741,0.9727,0.9660,0.9684,0.9672,0.9458,0.9458,4.5143
6,Gradient Boosting Classifier,0.9661,0.9947,0.9348,0.9784,0.9561,0.9285,0.9291,61.6044
7,Ada Boost Classifier,0.9615,0.9934,0.9374,0.9641,0.9506,0.9190,0.9193,13.0440
8,Quadratic Discriminant Analysis,0.9277,0.9649,0.8515,0.9612,0.9031,0.8458,0.8498,0.4976
9,Linear Discriminant Analysis,0.9216,0.9797,0.8136,0.9855,0.8913,0.8308,0.8402,0.9253


In [19]:
blended = blend_models(estimator_list = best_7, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9872,0.9993,0.9741,0.9933,0.9836,0.9731,0.9732
1,0.9877,0.9993,0.9767,0.9921,0.9844,0.9743,0.9743
2,0.9883,0.9992,0.9758,0.9944,0.9850,0.9754,0.9755
3,0.9875,0.9992,0.9747,0.9936,0.9841,0.9739,0.9740
4,0.9878,0.9994,0.9758,0.9933,0.9845,0.9745,0.9746
Mean,0.9877,0.9993,0.9754,0.9934,0.9843,0.9742,0.9743
SD,0.0004,0.0001,0.0009,0.0007,0.0005,0.0007,0.0007


In [20]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9885,0.9993,0.9763,0.9946,0.9853,0.9759,0.976


In [22]:
final_model = finalize_model(blended)

In [23]:
predictions = predict_model(final_model, data = test)

In [24]:
predictions

,date,ord,num_words_title,num_words_content,title_num_unique,content_num_unique,num_punctuations_title,num_punctuations_content,title_startswith_[,content_startswith_[,...,svd_word_content_12,svd_word_content_13,svd_word_content_14,svd_word_content_15,svd_word_content_16,svd_word_content_17,svd_word_content_18,svd_word_content_19,Label,Score
0,20200118,1,18,21,18,19,4,3,True,False,...,0.004437,0.006408,-0.000253,0.000488,0.000404,0.001396,-0.002347,-0.002453,0,0.0276
1,20200118,2,18,33,18,29,4,6,True,True,...,0.019293,0.018309,-0.000831,-0.004136,0.001645,0.004462,-0.013273,-0.016854,0,0.0011
2,20200118,3,18,28,18,25,4,0,True,False,...,0.002534,0.030085,0.023227,-0.003058,-0.000398,-0.001289,-0.000189,-0.003874,0,0.0075
3,20200118,4,18,37,18,31,4,0,True,False,...,0.002943,0.027705,-0.002326,0.005173,0.000858,0.004921,-0.006028,-0.007679,0,0.0009
4,20200118,5,18,48,18,43,4,4,True,False,...,0.004515,0.031615,-0.004745,-0.003062,0.001735,0.004303,-0.011496,0.001701,0,0.0085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142560,20200521,72,7,16,7,15,1,3,False,False,...,-0.005815,-0.023577,0.006968,0.001992,0.000316,0.005280,0.002878,-0.002281,1,0.9987
142561,20200521,73,7,10,7,9,1,2,False,False,...,-0.010156,-0.020302,0.006861,-0.008685,0.002474,0.000702,0.002855,-0.002565,1,0.9986
142562,20200521,74,7,10,7,9,1,2,False,False,...,-0.010156,-0.020302,0.006861,-0.008685,0.002474,0.000702,0.002855,-0.002565,1,0.9986
142563,20200521,75,7,11,7,11,1,0,False,False,...,-0.006017,-0.005691,0.001257,0.010712,-0.002057,0.006220,-0.000893,0.003121,1,0.9970


In [27]:
submission = pd.read_csv('sample_submission.csv') 

In [26]:
scores = predictions['Score']

In [28]:
scores = np.asarray(scores)

In [30]:
scores

array([0.0276, 0.0011, 0.0075, ..., 0.9986, 0.997 , 0.997 ])

In [31]:
pred_classes = np.where(scores > 0.5, 1, 0).reshape(-1)

In [33]:
submission['info'] = pred_classes

In [34]:
submission.head()

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0


In [35]:
submission.to_csv('pycaret_naive_softblending.csv',index=False)